In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

from sentinelhub import BBoxSplitter, BBox, CRS, CustomUrlParam
from eolearn.core import EOTask, EOPatch, LinearWorkflow, EOWorkflow, Dependency, FeatureType, OverwritePermission, LoadFromDisk, SaveToDisk
from eolearn.io import S2L1CWCSInput
from eolearn.geometry import VectorToRaster

import datetime

In [ ]:
from eopatches import get_bbox_splitter, get_bbox_gdf
from workflows import get_create_and_add_lpis_workflow
from plots import draw_bbox, draw_vector_timeless
from utilities import get_slovenia_crop_geopedia_idx_to_crop_id_mapping, get_group_id
from tasks import FixLPIS


## Load country

In [ ]:
country = gpd.read_file('./geojson/svn_buffered.geojson')
country_shape = country.geometry.values.tolist()[-1]
country_crs = CRS.WGS84

In [ ]:
number_of_squares = 3
bbox_splitter = BBoxSplitter([country_shape], country_crs, (25 * number_of_squares, 17 * number_of_squares))

bbox_list = bbox_splitter.get_bbox_list()
info_list = bbox_splitter.get_info_list()

geometry = [Polygon(bbox.get_polygon()) for bbox in bbox_splitter.bbox_list]
idxs_x = [info['index_x'] for info in bbox_splitter.info_list]
idxs_y = [info['index_y'] for info in bbox_splitter.info_list]

df = pd.DataFrame({'index_x':idxs_x, 'index_y':idxs_y})
gdf = gpd.GeoDataFrame(df, crs={'init':CRS.ogc_string(bbox_splitter.bbox_list[0].crs)}, geometry=geometry)

In [ ]:
ids = 1000
#figure
fig, ax = plt.subplots(figsize=(15, 15))
gdf.plot(ax=ax,facecolor='w',edgecolor='r',alpha=0.5)
country.plot(ax=ax, facecolor='w',edgecolor='b',alpha=0.5)
gdf[ids:ids+1].plot(ax=ax,facecolor='r',edgecolor='r',alpha=0.5)

plt.axis('off')

In [ ]:
gdf[10:11]

## Get data

In [ ]:
get_slovenia_crop_geopedia_idx_to_crop_id_mapping()

In [ ]:
country_name = 'Slovenia'
year = 2017
out_path = './eopatches'

In [ ]:
workflow_data = get_create_and_add_lpis_workflow(country_name, year, out_path)

In [ ]:
tasks = workflow_data.get_tasks()
create = tasks['create']
save = tasks['save']

In [ ]:
tasks

In [ ]:
bbox = np.array(bbox_list)[1000]
patch_name = 'patch'

workflow_data.execute({create:{'bbox':bbox},
                    save:{'eopatch_folder':patch_name}
                   })

In [ ]:
path_out = str(out_path + '/' + country_name + '/' + str(year))
path_of_patch = str(out_path + '/' + country_name + '/' + str(year) + '/' + patch_name)
name_of_feature = 'LPIS_2017'

In [ ]:
eopatch = EOPatch.load(path_of_patch)

In [ ]:
eopatch.scalar_timeless['FIELD_AREA_RATIO']

In [ ]:
fields = eopatch.vector_timeless[name_of_feature]

In [ ]:
fields.head(3)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
fields.plot(ax=ax,facecolor='b',edgecolor='b',alpha=0.5)

In [ ]:
fixlpis = FixLPIS(feature=name_of_feature,country=country_name)
patch = fixlpis.execute(eopatch)

patch.vector_timeless[name_of_feature].head(3)

## Group classes

In [ ]:
slo_crop_def = pd.read_excel('./WP2/SLO_LPIS_grouping.xlsx')

In [ ]:
slo_crop_def.rename(index=str, columns={"Group 1": "GROUP_1", "SIFKMRS": "CROP_ID"}, inplace=True)

In [ ]:
slo_crop_def['CROP_ID'][99] = '1204'

In [ ]:
slo_crops = slo_crop_def.CROP_ID.values.astype(int)
slo_groups = slo_crop_def.GROUP_1.values
slo_unique_groups = np.unique(slo_crop_def.GROUP_1.values)

In [ ]:
crops_to_groups = dict(zip(slo_crops, slo_groups))

In [ ]:
groups_to_number =  dict(zip(slo_unique_groups, range(len(slo_unique_groups))))

In [ ]:
crops_to_number = dict(zip(slo_crops, [groups_to_number[i] for i in slo_groups]))

## Workflow all

In [ ]:
tasks = workflow_data.get_tasks()
create = tasks['create']
add_lpis = tasks['add_lpis']
area_ratio = tasks[' area_ratio']
save = tasks['save']
fixlpis = FixLPIS(feature=name_of_feature,country=country_name)

add_data = S2L1CWCSInput(
    layer='BANDS-S2-L1C', 
    feature=(FeatureType.DATA, 'bands'),
    resx='10m', 
    resy='10m', 
    maxcc=0.80,
    time_difference=datetime.timedelta(days=1)
)

In [ ]:
class AddGroup(EOTask):
    def __init__(self, dictionary=crops_to_number, name_of_feature='LPIS_2017'):
        self.name_of_feature = name_of_feature
        self.dictionary = crops_to_number
        
    def execute(self, eopatch):
        gdf = eopatch.vector_timeless[self.name_of_feature]
        gdf['GROUP'] = [crops_to_number[i] for i in gdf.SIFKMRS]
        eopatch.vector_timeless[self.name_of_feature] = gdf
        return eopatch
    
add_group = AddGroup()

In [ ]:
rasterize = VectorToRaster(vector_input=(FeatureType.VECTOR_TIMELESS, name_of_feature),
                           raster_feature=(FeatureType.MASK_TIMELESS, name_of_feature),
                           values=None,
                           values_column='GROUP',
                           raster_shape=(FeatureType.DATA,'bands'),
                           raster_dtype=np.int16,
                           no_data_value=np.nan            
                          )

In [ ]:
workflow = LinearWorkflow(
    create,
    add_lpis,
    area_ratio,
    fixlpis,
    add_group,
    add_data,
    rasterize,
    save
)

In [ ]:
bbox = np.array(bbox_list)[1000]
patch_name = 'patch'
time_interval = ['2017-01-01', '2017-12-31'] 

workflow.execute({create:{'bbox':bbox},
                  add_data:{'time_interval': time_interval},
                  save:{'eopatch_folder':patch_name}
                 })

eopatch = EOPatch.load(path_of_patch)

In [ ]:
import matplotlib

cmap = matplotlib.colors.ListedColormap ( np.random.rand( 23,3))
#cmap = matplotlib.colors.ListedColormap(colors)

n_time = 5
image = np.clip(eopatch.data['bands'][n_time][..., [3, 2, 1]] * 3.5, 0, 1)
mask = np.squeeze(eopatch.mask_timeless[name_of_feature])

fig, ax0 = plt.subplots(figsize=(15, 15))
ax0.imshow(image)
ax0.imshow(mask, cmap=cmap, alpha=1)


In [ ]:
eopatch